In [2]:
import tensorflow as tf
import numpy as np

print(tf.__version__)
tf.random.set_seed(777)

2.1.0


In [16]:
x_data = [[1, 2, 1, 1],
          [2, 1, 3, 2],
          [3, 1, 3, 4],
          [4, 1, 5, 5],
          [1, 7, 5, 5],
          [1, 2, 5, 6],
          [1, 6, 6, 6],
          [1, 7, 7, 7]]
y_data = [[0, 0, 1],
          [0, 0, 1],
          [0, 0, 1],
          [0, 1, 0],
          [0, 1, 0],
          [0, 1, 0],
          [1, 0, 0],
          [1, 0, 0]]

#convert into numpy and float format
x_data = np.asarray(x_data, dtype=np.float32)
y_data = np.asarray(y_data, dtype=np.float32)

### one-hot encoding

Loss function 계산시, 값이 정수인코딩이라면, 클래스간의 관계가 균등하지 않다.
클래스의 표현 방법이 무작위성을 가진다는 것을 표현 가능

자연어 처리등에서 단어의 유사성을 구할 수 없다는 단점.


In [17]:

#dataset을 선언합니다.
# dataset = tf.data.Dataset.from_tensor_slices((x_data, y_data))
# dataset = dataset.repeat().batch(2)

nb_classes = 3 #class의 개수입니다.... 결과? 결과 개수인가

print(x_data.shape)
print(y_data.shape)

(8, 4)
(8, 3)


In [18]:
#Weight and bias setting
W = tf.Variable(tf.random.normal((4, nb_classes)), name='weight')
b = tf.Variable(tf.random.normal((nb_classes,)), name='bias')
variables = [W, b]

print(W,b)

<tf.Variable 'weight:0' shape=(4, 3) dtype=float32, numpy=
array([[-0.19601157,  0.21892233, -0.69331926],
       [-0.7571666 ,  0.62700933,  0.7786061 ],
       [-1.2000623 , -0.20798121, -2.2410874 ],
       [-0.48892695, -1.3168424 ,  1.5276555 ]], dtype=float32)> <tf.Variable 'bias:0' shape=(3,) dtype=float32, numpy=array([ 0.14128284, -1.2531738 , -0.14682682], dtype=float32)>


In [19]:
# tf.nn.softmax computes softmax activations
# softmax = exp(logits) / reduce_sum(exp(logits), dim)
def hypothesis(X):
    return tf.nn.softmax(tf.matmul(X, W) + b)

print(hypothesis(x_data))

tf.Tensor(
[[2.92883683e-02 2.06476226e-01 7.64235377e-01]
 [7.87439570e-02 6.69278800e-01 2.51977295e-01]
 [8.82659014e-03 2.16904264e-02 9.69483018e-01]
 [1.32640889e-02 1.56858221e-01 8.29877734e-01]
 [3.56227133e-07 4.90666367e-03 9.95092988e-01]
 [1.02955586e-04 6.11576368e-04 9.99285519e-01]
 [6.25253506e-07 2.54256395e-03 9.97456849e-01]
 [5.08256583e-08 9.72152979e-04 9.99027848e-01]], shape=(8, 3), dtype=float32)


In [20]:
# Softmax onehot test
sample_db = [[8,2,1,4]]
sample_db = np.asarray(sample_db, dtype=np.float32)

print(hypothesis(sample_db))

tf.Tensor([[0.00283937 0.03048898 0.9666716 ]], shape=(1, 3), dtype=float32)


In [21]:
def cost_fn_1(X, Y):
    logits = hypothesis(X)
    cost = -tf.reduce_sum(Y * tf.math.log(logits), axis=1)
    cost_mean = tf.reduce_mean(cost)
    
    return cost_mean

print(cost_fn(x_data, y_data))

tf.Tensor(5.9159136, shape=(), dtype=float32)


### Cost function

![crossentropy](./img/ce.png)

y : 실제값
k : 클래스의 개수
yj : 실제값 원-핫 벡터의 j번째 인덱스
pj : 샘플 데이터가 j번째 클래스일 확률(y_hat j로 나타내기도 함) 

ex)c가 실제값 원-핫 벡터에서 1을 가진 원소의 인덱스라면 pc = 1은 y_hat이 y를 정확하게 예측한 경우이다. 위의 코스트 함수에 대입해보면 -1log(1) = 0 , cost는 0이 되므로 코스트 함수로 적합하다.

![crossentropy](./img/ce2.png)

소프트맥스 함수의 최종 비용함수에서 k가 2라고 가정하면 logistic 회귀의 비용함수와 같다.

In [22]:
x = tf.constant(3.0)
with tf.GradientTape() as tape:
    tape.watch(x)
    y = x * x # x^2
dy_dx = tape.gradient(y, x) # Will compute to 6.0
print(dy_dx)

tf.Tensor(6.0, shape=(), dtype=float32)


In [23]:
def grad_fn(X, Y):
    with tf.GradientTape() as tape:
        loss = cost_fn_1(X, Y)
        grads = tape.gradient(loss, variables)

        return grads

print(grad_fn(x_data, y_data))

[<tf.Tensor: shape=(4, 3), dtype=float32, numpy=
array([[-0.21669796, -0.46917862,  0.6858766 ],
       [-1.605047  , -1.0851988 ,  2.6902459 ],
       [-1.5801449 , -1.4858341 ,  3.0659792 ],
       [-1.5888717 , -1.6917062 ,  3.2805781 ]], dtype=float32)>, <tf.Tensor: shape=(3,), dtype=float32, numpy=array([-0.23372163, -0.24208292,  0.47580457], dtype=float32)>]


In [24]:
def fit(X, Y, epochs=2000, verbose=100):
    optimizer =  tf.keras.optimizers.SGD(learning_rate=0.1)

    for i in range(epochs):
        grads = grad_fn(X, Y)
        optimizer.apply_gradients(zip(grads, variables))
        if (i==0) | ((i+1)%verbose==0):
            print('Loss at epoch %d: %f' %(i+1, cost_fn_1(X, Y).numpy()))
            
fit(x_data, y_data)

Loss at epoch 1: 3.223562
Loss at epoch 100: 0.730418
Loss at epoch 200: 0.612726
Loss at epoch 300: 0.549639
Loss at epoch 400: 0.498597
Loss at epoch 500: 0.451360
Loss at epoch 600: 0.405371
Loss at epoch 700: 0.359657
Loss at epoch 800: 0.313971
Loss at epoch 900: 0.269458
Loss at epoch 1000: 0.236338
Loss at epoch 1100: 0.223562
Loss at epoch 1200: 0.213007
Loss at epoch 1300: 0.203368
Loss at epoch 1400: 0.194534
Loss at epoch 1500: 0.186408
Loss at epoch 1600: 0.178909
Loss at epoch 1700: 0.171969
Loss at epoch 1800: 0.165530
Loss at epoch 1900: 0.159539
Loss at epoch 2000: 0.153952


In [25]:
sample_data = [[2,1,3,2]] # answer_label [[0,0,1]]
sample_data = np.asarray(sample_data, dtype=np.float32)

a = hypothesis(sample_data)

print(a)
print(tf.argmax(a, 1)) #index: 2

tf.Tensor([[0.00146075 0.07815406 0.9203852 ]], shape=(1, 3), dtype=float32)
tf.Tensor([2], shape=(1,), dtype=int64)


In [26]:
b = hypothesis(x_data)
print(b)
print(tf.argmax(b, 1))
print(tf.argmax(y_data, 1)) # matches with y_data

tf.Tensor(
[[4.6473992e-06 1.3204942e-03 9.9867487e-01]
 [1.4607507e-03 7.8154080e-02 9.2038518e-01]
 [2.5665841e-08 1.5866157e-01 8.4133840e-01]
 [4.6157754e-07 8.5580277e-01 1.4419670e-01]
 [2.5473171e-01 7.3356521e-01 1.1703093e-02]
 [1.3420093e-01 8.6571240e-01 8.6664819e-05]
 [7.5719875e-01 2.4274853e-01 5.2737465e-05]
 [9.1697198e-01 8.3027042e-02 1.0227931e-06]], shape=(8, 3), dtype=float32)
tf.Tensor([2 2 2 1 1 1 0 0], shape=(8,), dtype=int64)
tf.Tensor([2 2 2 1 1 1 0 0], shape=(8,), dtype=int64)


In [27]:
class softmax_classifer(tf.keras.Model):
    def __init__(self, nb_classes):
        super(softmax_classifer, self).__init__()
        self.W = tf.Variable(tf.random.normal((4, nb_classes)), name='weight')
        self.b = tf.Variable(tf.random.normal((nb_classes,)), name='bias')
        
    def softmax_regression(self, X):
        return tf.nn.softmax(tf.matmul(X, self.W) + self.b)
    
    def cost_fn(self, X, Y):
        logits = self.softmax_regression(X)
        cost = tf.reduce_mean(-tf.reduce_sum(Y * tf.math.log(logits), axis=1))        
        return cost
    
    def grad_fn(self, X, Y):
        with tf.GradientTape() as tape:
            cost = self.cost_fn(x_data, y_data)
            grads = tape.gradient(cost, self.variables)            
            return grads
    
    def fit(self, X, Y, epochs=2000, verbose=500):
        optimizer =  tf.keras.optimizers.SGD(learning_rate=0.1)

        for i in range(epochs):
            grads = self.grad_fn(X, Y)
            optimizer.apply_gradients(zip(grads, self.variables))
            if (i==0) | ((i+1)%verbose==0):
                print('Loss at epoch %d: %f' %(i+1, self.cost_fn(X, Y).numpy()))
            
model = softmax_classifer(nb_classes)
model.fit(x_data, y_data)

Loss at epoch 1: 4.056787
Loss at epoch 500: 0.356833
Loss at epoch 1000: 0.223017
Loss at epoch 1500: 0.178147
Loss at epoch 2000: 0.148037


# softmax fancy version

In [5]:
##수식이 아니라 함수를 이용해서 구해본다.

#numpy를 이용해 데이터를 load할 수 있다.
xy = np.loadtxt('data-04-zoo.csv', delimiter=',', dtype = np.float32)

#데이터 전처리.
x_data = xy[:, 0:-1]
y_data = xy[:, -1]

nb_classes = 7

#y는 one-hot의 형태로 되어있지 않음.
#tf.one_hot()을 통해 one_hot의 형태로 변형 가능하다.
#input : Rank N -> output : Rank N+1
Y_one_hot = tf.one_hot(y_data.astype(np.int32), nb_classes)

print(x_data.shape, Y_one_hot.shape)

(101, 16) (101, 7)


#### 학습 함수

In [12]:
W = tf.Variable(tf.random.normal((16, nb_classes)), name = 'weight')
b = tf.Variable(tf.random.normal((nb_classes, )), name = 'bias')

variables = [W, b] #두가지를 업데이트할 variables따로 저장

#hypothesis와 login_fn을 나눠서 구현했다.

def logit_fn(X):
    return tf.matmul(X, W)+b

""" 위에서는 이렇게 했었음
    def hypothesis(X):
    return tf.nn.softmax(tf.matmul(X, W) + b)
    
    def cost_fn_1(X, Y):
    logits = hypothesis(X)
    cost = -tf.reduce_sum(Y * tf.math.log(logits), axis=1)
    cost_mean = tf.reduce_mean(cost)
    
    return cost_mean
"""
#cost에서는 logit만 필요로 하는데 hypo는 왜 구현?
#학습시 정확도를 맞추기 위한 용도.
def hypothesis(X):
    return tf.nn.softmax(logit_fn(X))

def cost_fn(X, Y):
    
    logits = logit_fn(X)
    cost_i = tf.keras.losses.categorical_crossentropy(y_true = Y, y_pred = logits, from_logits = True)
    
    cost = tf.reduce_mean(cost_i)
    return cost

#grads를 구하기 위해 GradientTape 활용
#loss를 구하기 위해 cost_fn활용
#loss를 기반으로variables의 W와 b업데이트
def grad_fn(X, Y):
    with tf.GradientTape() as tape:
        loss = cost_fn(X, Y)
        grads = tape.gradient(loss, variables)
        return grads

#정확도. 굳이 hypo구현한 이유

#logits 보다는 hypothesis를 통해서 높은 값을 선택 예측?

def prediction(X, Y):
    pred = tf.argmax(hypothesis(X), 1)
    correct_prediction = tf.equal(pred, tf.argmax(Y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    
    return accuracy


In [13]:
def fit(X, Y, epochs=1000, verbose=100):
    optimizer =  tf.keras.optimizers.SGD(learning_rate=0.1)

    for i in range(epochs):
        grads = grad_fn(X, Y)
        optimizer.apply_gradients(zip(grads, variables))
        if (i==0) | ((i+1)%verbose==0):
#             print('Loss at epoch %d: %f' %(i+1, cost_fn(X, Y).numpy()))
            acc = prediction(X, Y).numpy()
            loss = cost_fn(X, Y).numpy() 
            print('Steps: {} Loss: {}, Acc: {}'.format(i+1, loss, acc))

fit(x_data, Y_one_hot)

Steps: 1 Loss: 6.631802082061768, Acc: 0.10891088843345642
Steps: 100 Loss: 0.5726618766784668, Acc: 0.8514851331710815
Steps: 200 Loss: 0.3897223472595215, Acc: 0.9009901285171509
Steps: 300 Loss: 0.3014775514602661, Acc: 0.9108911156654358
Steps: 400 Loss: 0.24435147643089294, Acc: 0.9306930899620056
Steps: 500 Loss: 0.20434370636940002, Acc: 0.9405940771102905
Steps: 600 Loss: 0.17501817643642426, Acc: 0.9702970385551453
Steps: 700 Loss: 0.1527751386165619, Acc: 0.9900990128517151
Steps: 800 Loss: 0.13542766869068146, Acc: 0.9900990128517151
Steps: 900 Loss: 0.12156669050455093, Acc: 0.9900990128517151
Steps: 1000 Loss: 0.11025161296129227, Acc: 0.9900990128517151
